<a href="https://colab.research.google.com/github/DipankarJDutta/DipankarJDutta-Biomarker-discovery-from-B-T-cells-in-FASD-GDM-with-LSTM/blob/Feature-extraction-with-Shap/LSTM_B_T_Median_optimized%20model_SHAP%20feature%20extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Fixing random seed to 007 for reproducibility
from numpy.random import seed
seed(7)
from tensorflow import set_random_seed
set_random_seed(7)

In [0]:
#Installing SHAP (SHapley Additive exPlanations)
!pip install shap

In [0]:
#Import SHAP specific essentials
from __future__ import print_function
from keras.preprocessing import sequence
from keras.layers import Embedding


In [0]:
# Loading essentials
from numpy import loadtxt
from numpy import reshape
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [0]:
# Load training data
x_train = loadtxt('B-T-median.csv', delimiter = ',')
x_test = loadtxt('B-T-median.csv', delimiter = ',')

In [0]:
# Reshape 2D training dataset into a 3D dataset with columns as features with one time-step
x_train = x_train.reshape(56, 1, 30)
x_test = x_test.reshape(56, 1, 30)

# Print dataset shape
x_train.shape
x_test.shape

(56, 1, 30)

In [0]:
#Specify binary output of good (1) and bad (0) learners, y_train and y_test, in x_train and x_test respectively. Learner type differentiated by population median.
y_train = x_train [:, :, -1]
y_test = x_test [:, :, -1]


In [0]:
# Define LSTM One-to-One Model with 3 LSTM cells (determined via hyperparameter optimization). Sigmoid activation on the output layer for binary classification.
model = Sequential()
model.add(LSTM(3, input_shape=(1,30)))
model.add(Dense(1, activation='sigmoid'))

In [0]:
# Compile the model 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
#Fit the model with all of the data as train and test samples
model.fit(x_train, y_train, batch_size = 8, epochs = 1000, shuffle = True, validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test, batch_size=8)
print('Test score:', score)
print('Test accuracy:', acc)

Train on 56 samples, validate on 56 samples
Epoch 1/1000
56/56 [==============================] - 5s 87ms/step - loss: 0.7107 - acc: 0.4464 - val_loss: 0.7086 - val_acc: 0.4464
Epoch 2/1000
56/56 [==============================] - 0s 732us/step - loss: 0.7075 - acc: 0.4464 - val_loss: 0.7059 - val_acc: 0.4464
Epoch 3/1000
56/56 [==============================] - 0s 586us/step - loss: 0.7051 - acc: 0.4464 - val_loss: 0.7033 - val_acc: 0.4464
Epoch 4/1000
56/56 [==============================] - 0s 601us/step - loss: 0.7023 - acc: 0.4464 - val_loss: 0.7012 - val_acc: 0.4464
Epoch 5/1000
56/56 [==============================] - 0s 592us/step - loss: 0.7008 - acc: 0.4464 - val_loss: 0.6989 - val_acc: 0.4464
Epoch 6/1000
56/56 [==============================] - 0s 667us/step - loss: 0.6982 - acc: 0.4464 - val_loss: 0.6972 - val_acc: 0.4464
Epoch 7/1000
56/56 [==============================] - 0s 649us/step - loss: 0.6966 - acc: 0.4464 - val_loss: 0.6954 - val_acc: 0.4464
Epoch 8/1000
56/56 

In [0]:
# Explain model with DeepExplainer from SHAP
#Get Shap values
import shap
explainer = shap.DeepExplainer(model, x_train)
shap_values = explainer.shap_values(x_test)

In [16]:
# Display shap values in the list
import numpy as np
np.set_printoptions(threshold=np.inf)
print(*shap_values, sep = ",")

[[[-2.50972488e-03 -2.27335725e-03 -1.52116319e-04  1.27375898e-03
   -8.88691022e-05  7.96753232e-04  3.97022944e-02  1.01041874e-03
   -6.06013562e-04  2.38698218e-03  2.29526041e-03 -2.98299050e-03
   -1.00131734e-03 -2.09240834e-03  6.41086785e-04  2.16409256e-02
   -2.54075136e-05 -2.55720478e-05  2.77379377e-03 -3.21489687e-03
   -2.45467899e-03  2.76588904e-02 -1.99548066e-04  4.89930598e-06
    4.40983833e-04  2.16169661e-04 -2.32360290e-03 -3.34572058e-03
   -5.05676271e-03  3.87364656e-01]]

 [[-3.19784096e-04 -4.44622605e-04 -1.65737143e-04  1.03069628e-03
    6.62440861e-05  3.61363726e-04  4.32696806e-02  9.23069642e-04
    4.35722611e-04  3.68768815e-03  2.03070698e-03 -1.25120813e-02
    1.03408275e-03 -1.83154095e-03 -5.83997615e-05  1.08692777e-02
   -1.09899611e-03  1.34579963e-03  5.38913530e-03  1.06080977e-02
    2.14989207e-02 -2.78656860e-02  2.34409718e-03 -7.19689137e-04
   -1.16392106e-04 -9.41289373e-03  7.63830980e-03 -5.57176379e-03
    2.15138098e-02  3.85

In [0]:
# Python object type - Shap_values
type(shap_values)

# Python object shape - Shap_values
nrow = len(shap_values)
ncol = len(shap_values[0])
print(nrow)
print(ncol)

1
56
